In [22]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [23]:
# Set seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [24]:
# Load the historical data
data = pd.read_csv('historical_data.csv')
data['Date'] = pd.to_datetime(data['Date'])  # Ensure date format

In [25]:
# List of tickers to process
tickers = ['TSLA', 'AAPL', 'AXP', 'TMUS']

In [26]:
# Store metrics for each ticker
results = {}

In [27]:
for ticker in tickers:
    print(f"\n🔹 Processing {ticker}...\n")
    
    # Step 1: Filter data for the current ticker
    stock_data = data[data['Ticker'] == ticker].copy()
    stock_data.sort_values('Date', inplace=True)
    
    # Step 2: Select relevant columns
    stock_data = stock_data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    
    # Step 3: Scale features
    scaler_features = MinMaxScaler(feature_range=(0, 1))
    numerical_features = ['Open', 'High', 'Low', 'Volume']
    stock_data[numerical_features] = scaler_features.fit_transform(stock_data[numerical_features])

    # Separate scaler for 'Close' price
    scaler_close = MinMaxScaler(feature_range=(0, 1))
    stock_data[['Close']] = scaler_close.fit_transform(stock_data[['Close']])

    # Step 4: Preprocess the data
    def preprocess_data(data, n_steps=10):
        X, y = [], []
        feature_columns = numerical_features + ['Close']
        values = data[feature_columns].values
        
        for i in range(n_steps, len(values)):
            X.append(values[i - n_steps:i, :])  # Last n_steps rows as features
            y.append(values[i, -1])  # Predict 'Close' price

        return np.array(X), np.array(y)

    X, y = preprocess_data(stock_data, n_steps=10)

    # Step 5: Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 6: Build the LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)
    ])

    # Step 7: Compile the model
    model.compile(optimizer='adam', loss='mse')

    # Step 8: Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

    # Step 9: Make predictions
    predictions = model.predict(X_test)

    # Step 10: Inverse scale the predictions and actual values
    y_test_scaled = scaler_close.inverse_transform(y_test.reshape(-1, 1)).flatten()
    predictions_scaled = scaler_close.inverse_transform(predictions.reshape(-1, 1)).flatten()

    # Step 11: Evaluate the model
    mae = mean_absolute_error(y_test_scaled, predictions_scaled)
    mape = mean_absolute_percentage_error(y_test_scaled, predictions_scaled)

    # Define accuracy threshold (5% tolerance)
    threshold_percentage = 5  
    percentage_errors = np.abs((y_test_scaled - predictions_scaled) / y_test_scaled) * 100
    acc = np.mean(percentage_errors <= threshold_percentage) * 100

    # Alternative accuracy metric
    acc2 = (1 - mape) * 100

    # Store results
    results[ticker] = {
        'MAE': mae,
        'MAPE (%)': mape * 100,
        'Accuracy (5% threshold)': acc,
        'Overall Accuracy (%)': acc2
    }


🔹 Processing TSLA...



c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - loss: 0.2741 - val_loss: 0.0440
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0379 - val_loss: 0.0287
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0224 - val_loss: 0.0238
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0218 - val_loss: 0.0129
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0192 - val_loss: 0.0120
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0137 - val_loss: 0.0104
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0129 - val_loss: 0.0096
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0115 - val_loss: 0.0089
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0092 - val_loss: 0.0090
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0099 - val_loss: 0.0085
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0099 - val_loss: 0.0087
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0091 - val_l

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.2536 - val_loss: 0.0161
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0258 - val_loss: 0.0176
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0137 - val_loss: 0.0134
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0155 - val_loss: 0.0137
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0119 - val_loss: 0.0079
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0091 - val_loss: 0.0072
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0080 - val_loss: 0.0082
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0084 - val_loss: 0.0066
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0118 - val_loss: 0.0079
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0086 - val_loss: 0.0065
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0086 - val_loss: 0.0076
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0095 - val_lo

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.1880 - val_loss: 0.0278
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0324 - val_loss: 0.0218
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0196 - val_loss: 0.0212
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0175 - val_loss: 0.0266
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0184 - val_loss: 0.0193
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0166 - val_loss: 0.0207
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0140 - val_loss: 0.0185
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0137 - val_loss: 0.0183
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0154 - val_loss: 0.0170
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0138 - val_loss: 0.0161
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0152 - val_loss: 0.0152
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0132 - val_loss: 0.0155
E

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.4531 - val_loss: 0.1556
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1349 - val_loss: 0.0596
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0488 - val_loss: 0.0150
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0269 - val_loss: 0.0165
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0279 - val_loss: 0.0208
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0223 - val_loss: 0.0159
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0178 - val_loss: 0.0150
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0189 - val_loss: 0.0156
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0192 - val_loss: 0.0156
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0177 - val_loss: 0.0140
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0190 - val_loss: 0.0145
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0175 - val_loss: 0.0140
E

In [29]:
# Step 12: Print final results
print("Model Performance Metrics:")

for ticker, metrics in results.items():
    # Get highest and lowest close prices from the original dataset
    stock_data = data[data['Ticker'] == ticker]  # Filter data for the current ticker
    highest_close = stock_data['Close'].max()
    lowest_close = stock_data['Close'].min()

    print(f"\n{ticker} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

    # Print highest and lowest closing prices
    print(f"Highest Closing Price: ${highest_close:.2f}")
    print(f"Lowest Closing Price: ${lowest_close:.2f}")


Model Performance Metrics:

TSLA Performance:
MAE: 9.73
MAPE (%): 4.33
Accuracy (5% threshold): 64.94
Overall Accuracy (%): 95.67
Highest Closing Price: $317.54
Lowest Closing Price: $108.10

AAPL Performance:
MAE: 3.43
MAPE (%): 2.18
Accuracy (5% threshold): 92.21
Overall Accuracy (%): 97.82
Highest Closing Price: $196.45
Lowest Closing Price: $125.02

AXP Performance:
MAE: 2.95
MAPE (%): 1.91
Accuracy (5% threshold): 94.81
Overall Accuracy (%): 98.09
Highest Closing Price: $181.33
Lowest Closing Price: $134.91

TMUS Performance:
MAE: 2.17
MAPE (%): 1.54
Accuracy (5% threshold): 98.70
Overall Accuracy (%): 98.46
Highest Closing Price: $152.41
Lowest Closing Price: $121.73
